## sub-task 2 제출 스크립트
ETRI FASHION-HOW Season 4 task2 제출코드입니다.
#### 주의: 반드시 본 파일을 이용하여 제출을 수행해야 하며 파일의 이름은 task.ipynb로 유지되어야 합니다.
- 작성하신 추론용 코드(예: test.py)를 본 스크립트 내의 etri_task2_submit() 함수로 작동되게끔 삽입하는 것으로 결과 제출을 수행할 수 있습니다.
코드는 크게 4가지 파트로 구성되며, 해당 파트의 특성을 지켜서 내용을 편집하시면 되겠습니다.
1. 제출용 aifactory 라이브러리 설치
2. etri_task2_submit() 함수 편집 (추론 스크립트)
3. submit() 함수로 wrapping
4. if __name__ == "__main__" 조건문을 이용한 제출 수행

※ 가능하면 제출시에는 포함되어 있는 train data를 폴더에서 제외하고 제출하시는 편이 좋습니다.
- 파일 크기 감소 → 업로드 시간 감소 → 전체 추론 수행 시간 감소

### 1. 제출용 aifactory 라이브러리 설치
#### 결과 전송에 필요하므로 아래와 같이 aifactory 라이브러리가 반드시 최신버전으로 설치될 수 있게끔 합니다

### 2. etri_task2_submit() 함수 편집 (추론 스크립트)
#### etri_task2_submit() 편집 시 주의사항

1. 아래 etri_task2_submit() 함수 내에 전체 추론 실행 코드를 삽입하고 결과를 dataframe으로 return하게끔 구성
   - Baseline이 아닌 다른 모델을 사용하는 경우에도 동일
2. 함수 내에서는 import * 가 적용되지 않으므로 필요한 import object를 직접 입력
   - 반드시 함수 내에서 import가 이루어져야 합니다.
3. argparse 사용시 args, _ = parser.parse_known_args()로 인자 지정
   args = parser.parse_args()는 jupyter에서 오류가 발생합니다!!!
4. 모델 내부의 경로는 ./ 으로 경로를 지정합니다. (예: weight 파일 경로)
5. 데이터는 /aif/Dataset/ 경로 아래에 있습니다. (코드 내용 참조) 
6. return할 결과물과 양식에 유의합니다.

In [19]:
def etri_task2_submit():
    import subprocess
    import sys
    
    subprocess.check_call([sys.executable, "-m", "pip", "install", "timm==0.8.21.dev0"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pytorch-metric-learning==2.1.0"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "chardet==5.2.0"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "numpy==1.22.4"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "albumentations==1.3.1"])

    from _18color_classification import BaseMain

    cfg = {
            "mode" : "infer", #train, #infer
            
            "model_name" : "nf_seresnet101",#"resnetv2_101x1_bit", #"tf_efficientnetv2_m.in21k", #"swinv2_base_window12to16_192to256_22kft1k",
            #"tf_efficientnetv2_s.in21k",#"eva_large_patch14_196.in22k_ft_in1k",#"beit_base_patch16_224.in22k_ft_in22k",
            "num_classes" : 18,
            
            "learning_rate" : 5e-4,
            "focal_alpha" : 2,
            "focal_gamma" : 2,
            "resize" : 224,
            
            "data_train_path" : "./sub-task2/Dataset/Train",
            "data_train_csv_path" : "./sub-task2/Dataset/info_etri20_color_train.csv",
            "data_valid_path" : "./sub-task2/Dataset/Validation",
            "data_valid_csv_path" : "./sub-task2/Dataset/info_etri20_color_validation.csv",
            
            "data_infer_path" : "/aif/Dataset/Test/",
            "data_infer_csv_path" : "/aif/Dataset/info_etri20_color_test.csv",
            # "data_infer_path" : "../sub-task2/Dataset/Test_sample",
            # "data_infer_csv_path" : "../sub-task2/Dataset/info_etri20_color_test_sample.csv",
            
            "epochs" : 80,
            "batch_size" : 4,
            "num_worker" : 1,
            "early_stop_patient" : 10,
            
            "reuse" : False, #True, #False
            "weight_path" : "./models/SpatialAug_SmallSize_GridTTA/23E-val0.6235782979204032-nf_seresnet101.pth",
            
            "save_path" : "./sub-task2/ckpt/",
            "output_path" : "./sub-task2/output/",
            "log_path" : "./sub-task2/logging/",
            "device" : "cpu",#"cuda",
            
            "binary_mode" : False,
            
            "note" : "Cutmix 사용, FocalLoss 사용, Adam Optim 사용"
        } 

    cfg["shuffle"] = False

    base_main = BaseMain(**cfg)

    df = base_main.infer(**cfg)
   
    return df

### 3. submit() 함수로 wrapping
#### 반드시 아래와 같이 submit() 이라는 함수로 위에 정의된 etri_task2_submit 함수를 wrapping해야 합니다.

In [20]:
def submit():
    return etri_task2_submit

### 4. if name == "main" 조건문을 이용한 제출 수행
#### 아래와 같이 if __name__ == "__main__" 구문 내에서 제출 함수가 실행되게끔 합니다.
#### ※ task별, 참가자별로 key가 다릅니다. 잘못 입력하지 않도록 유의바랍니다.
- key는 각팀 팀장분에게 메일로 할당된 계정을 이용하여 task별로 확인하실 수 있습니다.

In [21]:
import aifactory.score as aif
import time
t = time.time()
if __name__ == "__main__":
    #-----------------------------------------------------#
    aif.submit(model_name="SpatialAug_SmallSize_GridTTA_nf_seresnet101",               # 본인의 모델명 입력(버전 관리에 용이하게끔 편의에 맞게 지정합니다)
               key="a5267026-3d4d-458a-bda4-fdc880fea39e",                    # 본인의 task key 입력
               func=submit                            # 3.에서 wrapping한 submit function
               )
    #-----------------------------------------------------#
    print(time.time() - t)

file : task.py
python
파일 전송 완료
score = 0.6133333333
825.7119765281677
